In [ ]:
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import load_model
import joblib
import mediapipe as mp

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten(
    ) if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten(
    ) if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten(
    ) if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten(
    ) if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
# Initialize Mediapipe
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Function to extract keypoints from a frame


def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import joblib
import mediapipe as mp
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the trained model
final_model = load_model('slt_model.h5')

# Load the scaler
scaler = joblib.load('scaler.joblib')

# Loading file
int_to_label = np.load('label_to_int.npy', allow_pickle=True).item()

# Open a connection to the webcam (change the index if you have multiple cameras)
cap = cv2.VideoCapture(0)

# Initialize Mediapipe
mp_holistic = mp.solutions.holistic

# Set the maximum sequence length and feature dimensions
max_length = 195
feature_dimensions = 1662


with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame")
            break

        # Make detections
        image, results = mp.mediapipe_detection(frame, holistic)

        draw_styled_landmarks(image, results)

        # Extract keypoints
        keypoints = extract_keypoints(results)

        print(keypoints)

        new_data_array = keypoints

        # Add batch dimension
        new_data_point = np.expand_dims(new_data_array, axis=0)

        # Reshape to 2D array (flatten the additional dimensions)
        new_data_point_2d = new_data_point.reshape(
            -1, new_data_point.shape[-1])

        # Use the scaler to transform the data
        new_data_scaled = scaler.transform(new_data_point_2d)

        # Ensure consistency in the number of features
        if len(keypoints) != feature_dimensions:
            print(
                "Number of features in the new data does not match the expected number.")
            continue

        new_data_padded = pad_sequences(
            [new_data_scaled], padding='post', maxlen=max_length, dtype='float32')

        # Verify the shape of new_data_padded
        print("Shape of new_data_padded:", new_data_padded.shape)

        # Make predictions using the trained model
        predictions = final_model.predict(new_data_padded)

        # Post-process the predictions (e.g., convert probabilities to labels)
        predicted_label = np.argmax(predictions)

        # Decode the integer label to the original label
        predicted_sign = int_to_label[predicted_label]

        print(predicted_sign)

        # Display the predicted sign on the frame
        cv2.putText(frame, f"Prediction: {predicted_sign}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Display the frame
        cv2.imshow('Real-Time SLT Prediction', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release the video capture object and close the OpenCV windows
cap.release()
cv2.destroyAllWindows()

AttributeError: module 'mediapipe' has no attribute 'mediapipe_detection'